In [1]:
import sys

!{sys.executable} -m pip install bs4 requests

In [2]:
import requests

url = 'https://www.makro.nl/vestigingen/delft'
headers = {
    'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582'
}

def get_page_response(page_url):
    response = requests.get(page_url, headers=headers)
    
    return response

get_page_response(url)

<Response [200]>

In [3]:
from bs4 import BeautifulSoup

def convert_price_to_number(price):
    return float(price.replace('€', '').replace(',', '.'))

def get_fuel_price():
    html = get_page_response(url).text
    soup = BeautifulSoup(html, 'html.parser')
    div = soup.find('div', text='Euro 95 (E10)').parent.parent.find('div', class_='field-price')

    return convert_price_to_number(div.text)

get_fuel_price()

2.249

In [4]:
from datetime import datetime
import os, time

file_name = 'data.csv'
# Scan 8 times per day
retry_delay = 60 * 60 * 24 / 8

def get_timestamp():
    return datetime.now()

def init_file():
    if not os.path.isfile(file_name):
        with open(file_name, "a") as f:
            f.write("%s,%s\n" % ("fuel_price", "timestamp"))

def write_to_csv():
    fuel_price = get_fuel_price()
    timestamp = get_timestamp()
    row = "%s,%s\n" % (fuel_price, timestamp)
    
    with open(file_name, "a") as f:
        f.write(row)

init_file()

while True:
    write_to_csv()
    time.sleep(1)

KeyboardInterrupt: 